In [ ]:
%pip install pandas scikit-learn xgboost openpyxl torch matplotlib seaborn

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np
import xgboost as xgb
from sklearn.neural_network import MLPRegressor
import matplotlib.pyplot as plt
import scipy.stats as stats
import datetime

# Ensure openpyxl is installed
try:
    import openpyxl
except ImportError:
    raise ImportError("Missing optional dependency 'openpyxl'. Install it using: pip install openpyxl")

# Load dataset
file_path = r".\Nix.xlsx"
df = pd.read_excel(file_path, sheet_name='Sheet1', engine='openpyxl')

# Defining color and spectral feature columns
color_columns = ["L*", "a*", "b*", "c", "h", "X", "Z", "sRGB R", "sRGB G", "sRGB B", "C", "M", "Y", "K"]
spectral_columns = [f"R{wavelength} nm" for wavelength in range(400, 710, 10)]
y = df["O.C (%)"]  # Target variable

# Splitting the dataset into 70% training and 30% validation
X_train_color, X_val_color, y_train, y_val = train_test_split(df[color_columns], y, test_size=0.3, random_state=42)
X_train_spectral, X_val_spectral, _, _ = train_test_split(df[spectral_columns], y, test_size=0.3, random_state=42)

# Standardizing the features
scaler_color = StandardScaler()
X_train_color_scaled = scaler_color.fit_transform(X_train_color)
X_val_color_scaled = scaler_color.transform(X_val_color)

scaler_spectral = StandardScaler()
X_train_spectral_scaled = scaler_spectral.fit_transform(X_train_spectral)
X_val_spectral_scaled = scaler_spectral.transform(X_val_spectral)

# Levene's test for homogeneity of variance
levene_stat, levene_p = stats.levene(y_train, y_val, center='mean')
print(f"Levene's Test for SOC Variance: Statistic = {levene_stat:.4f}, p-value = {levene_p:.4f}")
if levene_p < 0.05:
    print("Levene's test: Reject null hypothesis, variances are unequal between training and validation sets.")
else:
    print("Levene's test: Fail to reject null hypothesis, variances are equal between training and validation sets.")

# Define RPIQ calculation
def calculate_rpiq(y_true, y_pred):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    q1, q3 = np.percentile(y_true, [25, 75])
    rpiq = (q3 - q1) / rmse if rmse != 0 else np.inf
    return rpiq

# Bootstrapping function
def bootstrap_metrics(y_true, y_pred, n_iterations=1000):
    r2_scores = []
    rmse_scores = []
    n_samples = len(y_true)
    
    for _ in range(n_iterations):
        indices = np.random.choice(range(n_samples), size=n_samples, replace=True)
        y_true_sample = y_true.iloc[indices] if isinstance(y_true, pd.Series) else y_true[indices]
        y_pred_sample = y_pred[indices]
        r2_scores.append(r2_score(y_true_sample, y_pred_sample))
        rmse_scores.append(np.sqrt(mean_squared_error(y_true_sample, y_pred_sample)))
    
    return {
        'R² Mean': np.mean(r2_scores), 'R² Std': np.std(r2_scores),
        'RMSE Mean': np.mean(rmse_scores), 'RMSE Std': np.std(rmse_scores)
    }

# Initialize models and hyperparameter grids
models = {
    "Random Forest": {
        "model": RandomForestRegressor(random_state=42),
        "param_grid": {
            'n_estimators': [100, 200],
            'max_depth': [None, 10, 20, 30],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4],
            'bootstrap': [True, False]
        }
    },
    "Gradient Boosting": {
        "model": GradientBoostingRegressor(random_state=42),
        "param_grid": {
            'n_estimators': [100, 200, 300],
            'learning_rate': [0.01, 0.1, 0.2],
            'max_depth': [3, 5, 7],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4]
        }
    },
    "XGBoost": {
        "model": xgb.XGBRegressor(objective='reg:squarederror', random_state=42),
        "param_grid": {
            'n_estimators': [100, 300, 500],
            'learning_rate': [0.01, 0.1, 0.3],
            'max_depth': [3, 6, 9],
            'min_child_weight': [1, 3, 5],
            'subsample': [0.6, 0.8, 1.0]
        }
    },
    "Neural Network": {
        "model": MLPRegressor(activation='relu', solver='adam', random_state=42),
        "param_grid": {
            'hidden_layer_sizes': [(100, 50), (200, 100), (300, 150)],
            'learning_rate_init': [0.001, 0.01, 0.1],
            'max_iter': [500, 1000, 2000]
        }
    }
}

# Store results
results = {}
bootstrap_results = {}

for model_name, model_info in models.items():
    model = model_info["model"]
    param_grid = model_info["param_grid"]

    # Grid Search for Color Data
    grid_search_color = GridSearchCV(model, param_grid, cv=5, scoring='r2', n_jobs=-1)
    grid_search_color.fit(X_train_color_scaled, y_train)
    best_model_color = grid_search_color.best_estimator_
    
    y_pred_color_train = best_model_color.predict(X_train_color_scaled)
    y_pred_color_val = best_model_color.predict(X_val_color_scaled)
    
    r2_color_train = r2_score(y_train, y_pred_color_train)
    rmse_color_train = np.sqrt(mean_squared_error(y_train, y_pred_color_train))
    bias_color_train = np.mean(y_pred_color_train - y_train)
    rpiq_color_train = calculate_rpiq(y_train, y_pred_color_train)
    
    r2_color_val = r2_score(y_val, y_pred_color_val)
    rmse_color_val = np.sqrt(mean_squared_error(y_val, y_pred_color_val))
    bias_color_val = np.mean(y_pred_color_val - y_val)
    rpiq_color_val = calculate_rpiq(y_val, y_pred_color_val)
    
    # Bootstrap for Color Data
    bootstrap_color = bootstrap_metrics(y_val, y_pred_color_val)
    
    # Grid Search for Spectral Data
    grid_search_spectral = GridSearchCV(model, param_grid, cv=5, scoring='r2', n_jobs=-1)
    grid_search_spectral.fit(X_train_spectral_scaled, y_train)
    best_model_spectral = grid_search_spectral.best_estimator_
    
    y_pred_spectral_train = best_model_spectral.predict(X_train_spectral_scaled)
    y_pred_spectral_val = best_model_spectral.predict(X_val_spectral_scaled)
    
    r2_spectral_train = r2_score(y_train, y_pred_spectral_train)
    rmse_spectral_train = np.sqrt(mean_squared_error(y_train, y_pred_spectral_train))
    bias_spectral_train = np.mean(y_pred_spectral_train - y_train)
    rpiq_spectral_train = calculate_rpiq(y_train, y_pred_spectral_train)
    
    r2_spectral_val = r2_score(y_val, y_pred_spectral_val)
    rmse_spectral_val = np.sqrt(mean_squared_error(y_val, y_pred_spectral_val))
    bias_spectral_val = np.mean(y_pred_spectral_val - y_val)
    rpiq_spectral_val = calculate_rpiq(y_val, y_pred_spectral_val)
    
    # Bootstrap for Spectral Data
    bootstrap_spectral = bootstrap_metrics(y_val, y_pred_spectral_val)
    
    results[model_name] = {
        "Color R² (Train)": r2_color_train, "Color RMSE (Train)": rmse_color_train, 
        "Color Bias (Train)": bias_color_train, "Color RPIQ (Train)": rpiq_color_train,
        "Color R² (Val)": r2_color_val, "Color RMSE (Val)": rmse_color_val, 
        "Color Bias (Val)": bias_color_val, "Color RPIQ (Val)": rpiq_color_val,
        "Spectral R² (Train)": r2_spectral_train, "Spectral RMSE (Train)": rmse_spectral_train, 
        "Spectral Bias (Train)": bias_spectral_train, "Spectral RPIQ (Train)": rpiq_spectral_train,
        "Spectral R² (Val)": r2_spectral_val, "Spectral RMSE (Val)": rmse_spectral_val, 
        "Spectral Bias (Val)": bias_spectral_val, "Spectral RPIQ (Val)": rpiq_spectral_val
    }
    
    bootstrap_results[model_name] = {
        "Color R² Mean": bootstrap_color['R² Mean'], "Color R² Std": bootstrap_color['R² Std'],
        "Color RMSE Mean": bootstrap_color['RMSE Mean'], "Color RMSE Std": bootstrap_color['RMSE Std'],
        "Spectral R² Mean": bootstrap_spectral['R² Mean'], "Spectral R² Std": bootstrap_spectral['R² Std'],
        "Spectral RMSE Mean": bootstrap_spectral['RMSE Mean'], "Spectral RMSE Std": bootstrap_spectral['RMSE Std']
    }

# Convert results to DataFrame and save
results_df = pd.DataFrame.from_dict(results, orient='index')
output_filename = f"model_performance_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
results_df.to_csv(output_filename, index=True)
print(f"Model performance results saved to '{output_filename}'.")

# Convert bootstrap results to DataFrame and save
bootstrap_df = pd.DataFrame.from_dict(bootstrap_results, orient='index')
bootstrap_output_filename = f"bootstrap_results_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
bootstrap_df.to_csv(bootstrap_output_filename, index=True)
print(f"Bootstrap results saved to '{bootstrap_output_filename}'.")

# Correlation Analysis
color_corr = df[color_columns].corrwith(y)
spectral_corr = df[spectral_columns].corrwith(y)

# Plot Color Data Correlation
plt.figure(figsize=(8, 5))
color_corr.sort_values().plot(kind='barh', color=plt.cm.Blues(np.linspace(0.3, 1, len(color_corr))))
plt.xlabel("Correlation Coefficient", fontweight='bold')
plt.ylabel("Color Features", fontweight='bold')
plt.title("Correlation between Color Data and SOC", fontweight='bold')
plt.savefig("color_soc_correlation.png", dpi=300, bbox_inches='tight')
plt.show()

# Plot Spectral Data Correlation
plt.figure(figsize=(8, 5))
spectral_corr.sort_values().plot(kind='barh', color=plt.cm.Blues(np.linspace(0.3, 1, len(spectral_corr))))
plt.xlabel("Correlation Coefficient", fontweight='bold')
plt.ylabel("Spectral Features", fontweight='bold')
plt.title("Correlation between Spectral Data and SOC", fontweight='bold')
plt.savefig("spectral_soc_correlation.png", dpi=300, bbox_inches='tight')
plt.show()

# Plot SOC Distribution with Smooth Curve
plt.figure(figsize=(8, 5))
plt.hist(y, bins=20, color='darkgreen', edgecolor='black', alpha=0.7)
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
p = stats.gaussian_kde(y)(x)
plt.plot(x, p * len(y) * (xmax - xmin) / 20, color='black', linewidth=2)
plt.xlabel("SOC (%)", fontweight='bold')
plt.ylabel("Frequency", fontweight='bold')
plt.title("Distribution of SOC", fontweight='bold')
plt.savefig("soc_distribution.png", dpi=300, bbox_inches='tight')
plt.show()

# Prediction Plots for Random Forest (best performing model)
best_model_color = models["Random Forest"]["model"].set_params(**grid_search_color.best_params_)
best_model_color.fit(X_train_color_scaled, y_train)
y_pred_color_val = best_model_color.predict(X_val_color_scaled)

best_model_spectral = models["Random Forest"]["model"].set_params(**grid_search_spectral.best_params_)
best_model_spectral.fit(X_train_spectral_scaled, y_train)
y_pred_spectral_val = best_model_spectral.predict(X_val_spectral_scaled)

fig, axes = plt.subplots(1, 2, figsize=(12, 6))

# Color Data Plot
axes[0].scatter(y_val, y_pred_color_val, color='red', marker='^', s=100, label='Validation Samples')
axes[0].plot([min(y_val), max(y_val)], [min(y_val), max(y_val)], linestyle='--', color='blue', label='1:1 Line')
axes[0].plot(np.unique(y_val), np.poly1d(np.polyfit(y_val, y_pred_color_val, 1))(np.unique(y_val)), color='black', label='Regression Line')
axes[0].set_xlabel("Measured SOC (%)", fontweight='bold')
axes[0].set_ylabel("Predicted SOC (%)", fontweight='bold')
axes[0].set_title("Color Data Prediction (RF)", fontweight='bold')
axes[0].text(min(y_val), max(y_pred_color_val), f'R² = {r2_score(y_val, y_pred_color_val):.2f}', fontsize=12, fontweight='bold')
axes[0].legend()

# Spectral Data Plot
axes[1].scatter(y_val, y_pred_spectral_val, color='red', marker='^', s=100, label='Validation Samples')
axes[1].plot([min(y_val), max(y_val)], [min(y_val), max(y_val)], linestyle='--', color='blue', label='1:1 Line')
axes[1].plot(np.unique(y_val), np.poly1d(np.polyfit(y_val, y_pred_spectral_val, 1))(np.unique(y_val)), color='black', label='Regression Line')
axes[1].set_xlabel("Measured SOC (%)", fontweight='bold')
axes[1].set_ylabel("Predicted SOC (%)", fontweight='bold')
axes[1].set_title("Spectral Data Prediction (RF)", fontweight='bold')
axes[1].text(min(y_val), max(y_pred_spectral_val), f'R² = {r2_score(y_val, y_pred_spectral_val):.2f}', fontsize=12, fontweight='bold')
axes[1].legend()

plt.tight_layout()
plt.savefig("prediction_plots_rf.png", dpi=300, bbox_inches='tight')
plt.show()

# Save calibration and validation samples with predicted values for Random Forest
df_train = X_train_color.copy()
df_train["Measured SOC (%)"] = y_train
df_train["Predicted SOC (%)"] = best_model_color.predict(X_train_color_scaled)
df_train["Type"] = "Calibration"

df_val = X_val_color.copy()
df_val["Measured SOC (%)"] = y_val
df_val["Predicted SOC (%)"] = y_pred_color_val
df_val["Type"] = "Validation"

df_combined = pd.concat([df_train, df_val])
output_filename = f"calibration_validation_samples_rf_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
df_combined.to_csv(output_filename, index=False)
print(f"Calibration and validation samples with predicted values saved to '{output_filename}'.")


# GMM

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

# Define RPIQ calculation (reused from original code)
def calculate_rpiq(y_true, y_pred):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    q1, q3 = np.percentile(y_true, [25, 75])
    rpiq = (q3 - q1) / rmse if rmse != 0 else np.inf
    return rpiq

# Load dataset
file_path = r"./Nix.xlsx"
try:
    df = pd.read_excel(file_path, sheet_name='Sheet1', engine='openpyxl')
except FileNotFoundError:
    raise FileNotFoundError(f"Dataset file '{file_path}' not found.")
except ValueError:
    raise ValueError("Error reading 'Sheet1' from the Excel file. Check file format.")

# Define feature columns
color_columns = ["L*", "a*", "b*", "c", "h", "X", "Z", "sRGB R", "sRGB G", "sRGB B", "C", "M", "Y", "K"]
y = df["O.C (%)"]

# Split dataset
X_train_color, X_val_color, y_train, y_val = train_test_split(df[color_columns], y, test_size=0.3, random_state=42)

# Standardize training features
scaler_color = StandardScaler()
X_train_color_scaled = scaler_color.fit_transform(X_train_color)
X_train_color_scaled_df = pd.DataFrame(X_train_color_scaled, columns=color_columns, index=X_train_color.index)
train_df = pd.concat([X_train_color.reset_index(drop=True), y_train.reset_index(drop=True)], axis=1)
train_df.to_csv("train_df.csv", index=False)

# Baseline RF hyperparameter tuning
rf_model = RandomForestRegressor(random_state=42)
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}
grid_search_color = GridSearchCV(rf_model, param_grid, cv=5, scoring='r2', n_jobs=-1)
grid_search_color.fit(X_train_color_scaled, y_train)
best_rf_params = grid_search_color.best_params_
print(f"Best RF Hyperparameters: {best_rf_params}")

# Store results
results_gmm = []
synthetic_sizes = [1000, 2000, 3000, 4000, 5000]
oc_thresholds = list(range(4, 15))

# Prepare joint data for GMM
X_train_joint = pd.concat([X_train_color, y_train], axis=1)
X_train_joint_scaled = scaler_color.fit_transform(X_train_joint)
joint_columns = color_columns + ["O.C (%)"]

# Calculate original skewness and kurtosis
print(f"Original Training Skewness: {stats.skew(y_train):.2f}")
print(f"Original Training Kurtosis: {stats.kurtosis(y_train):.2f}")

for num_samples in synthetic_sizes:
    # Fit GMM with error handling
    try:
        gmm = GaussianMixture(n_components=10, random_state=42, covariance_type='full')
        gmm.fit(X_train_joint_scaled)
    except Exception as e:
        print(f"GMM fitting failed for {num_samples} samples: {e}")
        continue
    
    # Generate synthetic samples
    synthetic_samples = gmm.sample(num_samples)[0]
    # Inverse transform to original scale
    synthetic_df = pd.DataFrame(scaler_color.inverse_transform(synthetic_samples), columns=joint_columns)
    
    for upper_limit in oc_thresholds:
        # Filter synthetic samples
        synthetic_filtered = synthetic_df[(synthetic_df["O.C (%)"] >= 3) & (synthetic_df["O.C (%)"] <= upper_limit)]
        print(f"Synthetic Samples (3-{upper_limit}% SOC, {num_samples} samples): {len(synthetic_filtered)}")
        
        # Save specific dataset
        if upper_limit == 7 and num_samples == 5000:
            synthetic_filtered.to_csv("gmm-3-7-5000.csv", index=False)
        
        if not synthetic_filtered.empty:
            # Merge original and synthetic data
            train_data = pd.concat([X_train_joint, synthetic_filtered]).reset_index(drop=True)
            
            # Calculate skewness and kurtosis for augmented data
            print(f"Augmented Skewness (3-{upper_limit}% SOC, {num_samples} samples): {stats.skew(train_data['O.C (%)']):.2f}")
            print(f"Augmented Kurtosis (3-{upper_limit}% SOC, {num_samples} samples): {stats.kurtosis(train_data['O.C (%)']):.2f}")
            
            X_train_final = train_data[color_columns]
            y_train_final = train_data["O.C (%)"]
            X_val_final = X_val_color[color_columns]
            
            # Standardize
            scaler_final = StandardScaler()
            X_train_final_scaled = scaler_final.fit_transform(X_train_final)
            X_val_final_scaled = scaler_final.transform(X_val_final)
            
            # Train RF with tuned hyperparameters
            rf_model = RandomForestRegressor(**best_rf_params, random_state=42)
            rf_model.fit(X_train_final_scaled, y_train_final)
            y_pred = rf_model.predict(X_val_final_scaled)
            
            # Compute metrics
            r2 = r2_score(y_val, y_pred)
            rmse = np.sqrt(mean_squared_error(y_val, y_pred))
            bias = np.mean(y_pred - y_val)
            rpiq = calculate_rpiq(y_val, y_pred)
            
            results_gmm.append({
                "Sample Size": num_samples,
                "OC Range": f"3-{upper_limit}%",
                "R²": r2,
                "RMSE": rmse,
                "Bias": bias,
                "RPIQ": rpiq
            })

# Save results
df_gmm = pd.DataFrame(results_gmm)
output_path = f"GMM_Results_ALL_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx"
df_gmm.to_excel(output_path, index=False)
print(f"✅ GMM results saved successfully: {output_path}")

# Heatmap visualization for R²
pivot_r2 = df_gmm.pivot(index="Sample Size", columns="OC Range", values="R²")
plt.figure(figsize=(10, 6))
sns.heatmap(pivot_r2, annot=True, cmap='Blues', fmt='.2f')
plt.title("Validation R² for GMM-Augmented RF")
plt.savefig("gmm_r2_heatmap.png", dpi=300, bbox_inches='tight')
plt.show()

# Heatmap visualization for RMSE
pivot_rmse = df_gmm.pivot(index="Sample Size", columns="OC Range", values="RMSE")
plt.figure(figsize=(10, 6))
sns.heatmap(pivot_rmse, annot=True, cmap='Blues', fmt='.2f')
plt.title("Validation RMSE for GMM-Augmented RF")
plt.savefig("gmm_rmse_heatmap.png", dpi=300, bbox_inches='tight')
plt.show()

# Heatmap visualization for Bias
pivot_bias = df_gmm.pivot(index="Sample Size", columns="OC Range", values="Bias")
plt.figure(figsize=(10, 6))
sns.heatmap(pivot_bias, annot=True, cmap='Blues', fmt='.2f')
plt.title("Validation Bias for GMM-Augmented RF")
plt.savefig("gmm_bias_heatmap.png", dpi=300, bbox_inches='tight')
plt.show()

# Heatmap visualization for RPIQ
pivot_rpiq = df_gmm.pivot(index="Sample Size", columns="OC Range", values="RPIQ")
plt.figure(figsize=(10, 6))
sns.heatmap(pivot_rpiq, annot=True, cmap='Blues', fmt='.2f')
plt.title("Validation RPIQ for GMM-Augmented RF")
plt.savefig("gmm_rpiq_heatmap.png", dpi=300, bbox_inches='tight')
plt.show()

# Scatter plot for best GMM model (3-7% SOC, 5000 samples)
if (df_gmm["Sample Size"] == 5000) & (df_gmm["OC Range"] == "3-7%").any():
    best_gmm = df_gmm[(df_gmm["Sample Size"] == 5000) & (df_gmm["OC Range"] == "3-7%")]
    synthetic_filtered = pd.read_csv("gmm-3-7-5000.csv")
    train_data = pd.concat([X_train_joint, synthetic_filtered]).reset_index(drop=True)
    X_train_final = train_data[color_columns]
    y_train_final = train_data["O.C (%)"]
    X_val_final = X_val_color[color_columns]
    
    scaler_final = StandardScaler()
    X_train_final_scaled = scaler_final.fit_transform(X_train_final)
    X_val_final_scaled = scaler_final.transform(X_val_final)
    
    rf_model = RandomForestRegressor(**best_rf_params, random_state=42)
    rf_model.fit(X_train_final_scaled, y_train_final)
    y_pred = rf_model.predict(X_val_final_scaled)
    
    plt.figure(figsize=(6, 6))
    plt.scatter(y_val, y_pred, color='red', marker='o', s=100, label='Validation Samples')
    plt.plot([min(y_val), max(y_val)], [min(y_val), max(y_val)], linestyle='--', color='blue', label='1:1 Line')
    plt.plot(np.unique(y_val), np.poly1d(np.polyfit(y_val, y_pred, 1))(np.unique(y_val)), color='black', label='Regression Line')
    plt.xlabel("Measured SOC (%)", fontweight='bold')
    plt.ylabel("Predicted SOC (%)", fontweight='bold')
    plt.title("GMM-Augmented RF (3-7% SOC, 5000 samples)", fontweight='bold')
    plt.text(min(y_val), max(y_pred), f'R² = {r2_score(y_val, y_pred):.2f}', fontsize=12, fontweight='bold')
    plt.legend()
    plt.savefig("gmm_prediction_plot.png", dpi=300, bbox_inches='tight')
    plt.show()

# KS TEST

In [ ]:
import pandas as pd
import scipy.stats as stats
import datetime
import os

# Load datasets
try:
    gmm_data = pd.read_csv("gmm-3-7-5000.csv")
    gmm_train_data = pd.read_csv("train_df.csv")
except FileNotFoundError as e:
    print(f"Error: File not found - {e}")
    exit(1)

# Identify common columns
common_columns = [col for col in gmm_train_data.columns if col in gmm_data.columns]
if not common_columns:
    print("Error: No common columns found between the datasets.")
    exit(1)

# Check for missing values
if gmm_data[common_columns].isnull().any().any() or gmm_train_data[common_columns].isnull().any().any():
    print("Error: Missing values detected in one or both datasets. Please handle missing values before proceeding.")
    exit(1)

# Check for non-numeric columns
non_numeric_columns = [col for col in common_columns if not pd.api.types.is_numeric_dtype(gmm_train_data[col]) or not pd.api.types.is_numeric_dtype(gmm_data[col])]
if non_numeric_columns:
    print(f"Warning: Skipping non-numeric columns: {non_numeric_columns}")
    common_columns = [col for col in common_columns if col not in non_numeric_columns]

# Perform KS test and store results
results_ks = []
for column in common_columns:
    try:
        stat, p_value = stats.ks_2samp(gmm_train_data[column], gmm_data[column])
        result = {
            "Column": column,
            "KS Statistic": stat,
            "P-value": p_value,
            "Interpretation": "Similar (fail to reject H0)" if p_value > 0.05 else "Different (reject H0)"
        }
        results_ks.append(result)
        
        print(f"\nKS test for {column}:")
        print(f"  Statistic: {stat:.4f}")
        print(f"  P-value: {p_value:.4f}")
        print(f"  Interpretation: The distributions of {column} are {result['Interpretation'].lower()} (α=0.05)")
    except Exception as e:
        print(f"Error performing KS test for {column}: {e}")
        results_ks.append({
            "Column": column,
            "KS Statistic": None,
            "P-value": None,
            "Interpretation": f"Error: {str(e)}"
        })

# Save results to CSV
results_df = pd.DataFrame(results_ks)
output_filename = f"ks_test_results_all_variables_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
results_df.to_csv(output_filename, index=False)
print(f"\n✅ KS test results for all variables saved successfully to '{output_filename}'")

# GAN

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error
import tensorflow as tf
from tensorflow.keras import layers
import datetime

# Load dataset
file_path = r"./Nix.xlsx"
df = pd.read_excel(file_path, sheet_name='Sheet1', engine='openpyxl')

# Define feature columns
color_columns = ["L*", "a*", "b*", "c", "h", "X", "Z", "sRGB R", "sRGB G", "sRGB B", "C", "M", "Y", "K"]
y = df["O.C (%)"]

# Split dataset
X_train_color, X_val_color, y_train, y_val = train_test_split(df[color_columns], y, test_size=0.3, random_state=42)

# Prepare joint data for GAN (color features + SOC)
X_train_joint = pd.concat([X_train_color, y_train], axis=1)
joint_columns = color_columns + ["O.C (%)"]

# Standardize joint data
scaler_joint = StandardScaler()
X_train_joint_scaled = scaler_joint.fit_transform(X_train_joint)
X_train_joint_scaled_df = pd.DataFrame(X_train_joint_scaled, columns=joint_columns, index=X_train_color.index)
train_df = pd.concat([X_train_color.reset_index(drop=True), y_train.reset_index(drop=True)], axis=1)
train_df.to_csv("train_df.csv", index=False)

# Define GAN components
def build_generator(noise_dim=100, output_dim=len(joint_columns)):
    model = tf.keras.Sequential([
        layers.Dense(128, activation='relu', input_dim=noise_dim),
        layers.Dense(64, activation='relu'),
        layers.Dense(32, activation='relu'),
        layers.Dense(output_dim)
    ])
    return model

def build_discriminator(input_dim=len(joint_columns)):
    model = tf.keras.Sequential([
        layers.Dense(64, activation='relu', input_dim=input_dim),
        layers.Dense(32, activation='relu'),
        layers.Dense(16, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])
    return model

# Gradient penalty for Wasserstein GAN with Gradient Penalty (WGAN-GP)
def gradient_penalty(discriminator, real_data, fake_data):
    batch_size = tf.shape(real_data)[0]
    alpha = tf.random.uniform([batch_size, 1], 0.0, 1.0)
    interpolated = alpha * real_data + (1 - alpha) * fake_data
    with tf.GradientTape() as tape:
        tape.watch(interpolated)
        pred = discriminator(interpolated, training=True)
    grads = tape.gradient(pred, interpolated)
    norm = tf.sqrt(tf.reduce_sum(tf.square(grads), axis=1))
    gp = tf.reduce_mean((norm - 1.0) ** 2)
    return gp

# Loss functions
def discriminator_loss(real_output, fake_output, gp, gp_weight=10.0):
    real_loss = -tf.reduce_mean(real_output)
    fake_loss = tf.reduce_mean(fake_output)
    total_loss = real_loss + fake_loss + gp_weight * gp
    return total_loss

def generator_loss(fake_output):
    return -tf.reduce_mean(fake_output)

# Training step
@tf.function
def train_step(real_data, generator, discriminator, g_optimizer, d_optimizer, noise_dim, batch_size):
    noise = tf.random.normal([batch_size, noise_dim])
    
    with tf.GradientTape() as g_tape, tf.GradientTape() as d_tape:
        fake_data = generator(noise, training=True)
        real_output = discriminator(real_data, training=True)
        fake_output = discriminator(fake_data, training=True)
        gp = gradient_penalty(discriminator, real_data, fake_data)
        
        d_loss = discriminator_loss(real_output, fake_output, gp)
        g_loss = generator_loss(fake_output)
    
    g_gradients = g_tape.gradient(g_loss, generator.trainable_variables)
    d_gradients = d_tape.gradient(d_loss, discriminator.trainable_variables)
    
    g_optimizer.apply_gradients(zip(g_gradients, generator.trainable_variables))
    d_optimizer.apply_gradients(zip(d_gradients, discriminator.trainable_variables))
    
    return g_loss, d_loss

# Training function
def train_gan(generator, discriminator, data, epochs=10000, batch_size=64, noise_dim=100):
    g_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5)
    d_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5)
    
    dataset = tf.data.Dataset.from_tensor_slices(data).shuffle(1000).batch(batch_size)
    
    for epoch in range(epochs):
        for batch in dataset:
            g_loss, d_loss = train_step(batch, generator, discriminator, g_optimizer, d_optimizer, noise_dim, batch_size)
        
        if (epoch + 1) % 1000 == 0:
            print(f"Epoch {epoch + 1}/{epochs}, Generator Loss: {g_loss:.4f}, Discriminator Loss: {d_loss:.4f}")

# Initialize GAN
generator = build_generator()
discriminator = build_discriminator()

# Train GAN
train_gan(generator, discriminator, X_train_joint_scaled, epochs=10000, batch_size=64, noise_dim=100)

# Store results
results_gan = []
synthetic_sizes = [1000, 2000, 3000, 4000, 5000]
oc_thresholds = list(range(4, 15))

for num_samples in synthetic_sizes:
    # Generate synthetic samples
    noise = tf.random.normal([num_samples, 100])
    synthetic_samples = generator(noise, training=False).numpy()
    synthetic_df = pd.DataFrame(synthetic_samples, columns=joint_columns)
    
    # Inverse transform to original scale
    synthetic_df = pd.DataFrame(scaler_joint.inverse_transform(synthetic_samples), columns=joint_columns)
    
    for upper_limit in oc_thresholds:
        # Filter synthetic samples within SOC range [3, upper_limit]
        synthetic_filtered = synthetic_df[(synthetic_df["O.C (%)"] >= 3) & (synthetic_df["O.C (%)"] <= upper_limit)]
        
        # Save specific dataset for upper_limit=7 and num_samples=5000
        if upper_limit == 7 and num_samples == 5000:
            synthetic_filtered.to_csv("gan-3-7-5000.csv", index=False)
        
        # Train model if synthetic data is not empty
        if not synthetic_filtered.empty:
            # Merge original training data with synthetic data
            train_data = pd.concat([X_train_joint, synthetic_filtered]).reset_index(drop=True)
            
            X_train_final = train_data[color_columns]
            y_train_final = train_data["O.C (%)"]
            
            # Ensure validation features match training column order
            X_val_final = X_val_color[color_columns]
            
            # Standardize both train and validation sets
            scaler_final = StandardScaler()
            X_train_final_scaled = scaler_final.fit_transform(X_train_final)
            X_val_final_scaled = scaler_final.transform(X_val_final)
            
            # Train RandomForest model
            rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
            rf_model.fit(X_train_final_scaled, y_train_final)
            y_pred = rf_model.predict(X_val_final_scaled)
            
            # Compute performance metrics
            r2 = r2_score(y_val, y_pred)
            rmse = np.sqrt(mean_squared_error(y_val, y_pred))
            
            results_gan.append({
                "Sample Size": num_samples,
                "OC Range": f"3-{upper_limit}%",
                "R²": r2,
                "RMSE": rmse
            })

# Save results
df_gan = pd.DataFrame(results_gan)
output_path = f"GAN_Results_ALL_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx"
df_gan.to_excel(output_path, index=False)

print(f"✅ GAN results saved successfully: {output_path}")

# KNN

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.neighbors import NearestNeighbors
import datetime

# Load dataset
file_path = r"./Nix.xlsx"
df = pd.read_excel(file_path, sheet_name='Sheet1', engine='openpyxl')

# Define feature columns
color_columns = ["L*", "a*", "b*", "c", "h", "X", "Z", "sRGB R", "sRGB G", "sRGB B", "C", "M", "Y", "K"]
y = df["O.C (%)"]

# Split dataset
X_train_color, X_val_color, y_train, y_val = train_test_split(df[color_columns], y, test_size=0.3, random_state=42)

# Prepare joint data for KNN (color features + SOC)
X_train_joint = pd.concat([X_train_color, y_train], axis=1)
joint_columns = color_columns + ["O.C (%)"]

# Standardize joint data
scaler_joint = StandardScaler()
X_train_joint_scaled = scaler_joint.fit_transform(X_train_joint)
X_train_joint_scaled_df = pd.DataFrame(X_train_joint_scaled, columns=joint_columns, index=X_train_color.index)
train_df = pd.concat([X_train_color.reset_index(drop=True), y_train.reset_index(drop=True)], axis=1)
train_df.to_csv("train_df.csv", index=False)

# KNN synthetic data generation function
def generate_knn_synthetic_samples(data_scaled, n_samples, k=5, random_state=42):
    np.random.seed(random_state)
    nbrs = NearestNeighbors(n_neighbors=k, metric='euclidean').fit(data_scaled)
    synthetic_samples = []
    
    for _ in range(n_samples):
        # Randomly select a sample
        idx = np.random.randint(0, len(data_scaled))
        # Find K nearest neighbors
        distances, indices = nbrs.kneighbors([data_scaled[idx]])
        # Average the feature vectors of the K neighbors
        neighbor_samples = data_scaled[indices[0]]
        synthetic_sample = np.mean(neighbor_samples, axis=0)
        synthetic_samples.append(synthetic_sample)
    
    return np.array(synthetic_samples)

# Store results
results_knn = []
synthetic_sizes = [1000, 2000, 3000, 4000, 5000]
oc_thresholds = list(range(4, 15))

for num_samples in synthetic_sizes:
    # Generate synthetic samples
    synthetic_samples_scaled = generate_knn_synthetic_samples(X_train_joint_scaled, num_samples, k=5)
    
    # Inverse transform to original scale
    synthetic_samples = scaler_joint.inverse_transform(synthetic_samples_scaled)
    synthetic_df = pd.DataFrame(synthetic_samples, columns=joint_columns)
    
    for upper_limit in oc_thresholds:
        # Filter synthetic samples within SOC range [3, upper_limit]
        synthetic_filtered = synthetic_df[(synthetic_df["O.C (%)"] >= 3) & (synthetic_df["O.C (%)"] <= upper_limit)]
        
        # Save specific dataset for upper_limit=7 and num_samples=5000
        if upper_limit == 7 and num_samples == 5000:
            synthetic_filtered.to_csv("knn-3-7-5000.csv", index=False)
        
        # Train model if synthetic data is not empty
        if not synthetic_filtered.empty:
            # Merge original training data with synthetic data
            train_data = pd.concat([X_train_joint, synthetic_filtered]).reset_index(drop=True)
            
            X_train_final = train_data[color_columns]
            y_train_final = train_data["O.C (%)"]
            
            # Ensure validation features match training column order
            X_val_final = X_val_color[color_columns]
            
            # Standardize both train and validation sets
            scaler_final = StandardScaler()
            X_train_final_scaled = scaler_final.fit_transform(X_train_final)
            X_val_final_scaled = scaler_final.transform(X_val_final)
            
            # Train RandomForest model
            rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
            rf_model.fit(X_train_final_scaled, y_train_final)
            y_pred = rf_model.predict(X_val_final_scaled)
            
            # Compute performance metrics
            r2 = r2_score(y_val, y_pred)
            rmse = np.sqrt(mean_squared_error(y_val, y_pred))
            
            results_knn.append({
                "Sample Size": num_samples,
                "OC Range": f"3-{upper_limit}%",
                "R²": r2,
                "RMSE": rmse
            })

# Save results
df_knn = pd.DataFrame(results_knn)
output_path = f"KNN_Results_ALL_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx"
df_knn.to_excel(output_path, index=False)

print(f"✅ KNN results saved successfully: {output_path}")

# Bootstrapping

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error
import datetime

# Load dataset
file_path = r"./Nix.xlsx"
df = pd.read_excel(file_path, sheet_name='Sheet1', engine='openpyxl')

# Define feature columns
color_columns = ["L*", "a*", "b*", "c", "h", "X", "Z", "sRGB R", "sRGB G", "sRGB B", "C", "M", "Y", "K"]
y = df["O.C (%)"]

# Split dataset
X_train_color, X_val_color, y_train, y_val = train_test_split(df[color_columns], y, test_size=0.3, random_state=42)

# Prepare joint data for bootstrapping (color features + SOC)
X_train_joint = pd.concat([X_train_color, y_train], axis=1)
joint_columns = color_columns + ["O.C (%)"]

# Save original training data
train_df = pd.concat([X_train_color.reset_index(drop=True), y_train.reset_index(drop=True)], axis=1)
train_df.to_csv("train_df.csv", index=False)

# Bootstrapping synthetic data generation function
def generate_bootstrap_synthetic_samples(data, n_samples, noise_factor=0.05, random_state=42):
    np.random.seed(random_state)
    n_features = data.shape[1]
    synthetic_samples = []
    
    # Calculate standard deviation for each feature
    feature_stds = data.std().values
    
    # Generate synthetic samples
    for _ in range(n_samples):
        # Sample with replacement
        idx = np.random.choice(data.index, size=1)[0]
        sample = data.loc[idx].values
        # Add Gaussian noise (5% of each feature's standard deviation)
        noise = np.random.normal(0, noise_factor * feature_stds, size=n_features)
        synthetic_sample = sample + noise
        synthetic_samples.append(synthetic_sample)
    
    return pd.DataFrame(synthetic_samples, columns=data.columns)

# Store results
results_bootstrap = []
synthetic_sizes = [1000, 2000, 3000, 4000, 5000]
oc_thresholds = list(range(4, 15))

for num_samples in synthetic_sizes:
    # Generate synthetic samples
    synthetic_df = generate_bootstrap_synthetic_samples(X_train_joint, num_samples, noise_factor=0.05)
    
    for upper_limit in oc_thresholds:
        # Filter synthetic samples within SOC range [3, upper_limit]
        synthetic_filtered = synthetic_df[(synthetic_df["O.C (%)"] >= 3) & (synthetic_df["O.C (%)"] <= upper_limit)]
        
        # Save specific dataset for upper_limit=7 and num_samples=5000
        if upper_limit == 7 and num_samples == 5000:
            synthetic_filtered.to_csv("bootstrap-3-7-5000.csv", index=False)
        
        # Train model if synthetic data is not empty
        if not synthetic_filtered.empty:
            # Merge original training data with synthetic data
            train_data = pd.concat([X_train_joint, synthetic_filtered]).reset_index(drop=True)
            
            X_train_final = train_data[color_columns]
            y_train_final = train_data["O.C (%)"]
            
            # Ensure validation features match training column order
            X_val_final = X_val_color[color_columns]
            
            # Standardize both train and validation sets
            scaler_final = StandardScaler()
            X_train_final_scaled = scaler_final.fit_transform(X_train_final)
            X_val_final_scaled = scaler_final.transform(X_val_final)
            
            # Train RandomForest model
            rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
            rf_model.fit(X_train_final_scaled, y_train_final)
            y_pred = rf_model.predict(X_val_final_scaled)
            
            # Compute performance metrics
            r2 = r2_score(y_val, y_pred)
            rmse = np.sqrt(mean_squared_error(y_val, y_pred))
            
            results_bootstrap.append({
                "Sample Size": num_samples,
                "OC Range": f"3-{upper_limit}%",
                "R²": r2,
                "RMSE": rmse
            })

# Save results
df_bootstrap = pd.DataFrame(results_bootstrap)
output_path = f"Bootstrap_Results_ALL_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx"
df_bootstrap.to_excel(output_path, index=False)

print(f"✅ Bootstrap results saved successfully: {output_path}")